In [ ]:
from sqlalchemy import create_engine

server = 'localhost\\MSSQLSERVER2022' 
database = 'Financial_Transactions'

try:
  
    connection_string = (
        f"mssql+pyodbc://{server}/{database}?"
        "driver=ODBC+Driver+17+for+SQL+Server"
    )
    
   
    engine = create_engine(connection_string)
    connection = engine.connect()
    
    print("Connection successful!")
    
   
    connection.close()

except Exception as e:
   
    print("\n❌❌❌ فشل الاتصال! ❌❌❌")
    print("Connection failed!")
    print("\nالخطأ هو (The error is):")
    print(e)

--- الخطوة الأولى: اختبار الاتصال بالداتا بيز ---

✅✅✅ نجح الاتصال! ✅✅✅
Connection successful!


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import ijson
import gc
from itertools import islice




server = 'localhost\\MSSQLSERVER2022' 
database = 'Financial_Transactions'


try:
    connection_string = (
        f"mssql+pyodbc://{server}/{database}?"
        "driver=ODBC+Driver+17+for+SQL+Server"
    )
    engine = create_engine(connection_string)

    
    labels_file_path = r'D:\INSTANT\archive(1)\train_fraud_labels.json' #
    table_name = 'FraudLabels' 
    batch_size = 100_000  
    
    with open(labels_file_path, 'rb') as f:
        kv_generator = ijson.kvitems(f, 'target')
        
        is_first_batch = True
        while True:
           
            batch = list(islice(kv_generator, batch_size))
            if not batch:
                break 
                
            print(f"بعالج دفعة بحجم {len(batch)} صف...")
            
            temp_df = pd.DataFrame(batch, columns=['id', 'is_fraud_str'])
            temp_df['is_fraud'] = temp_df['is_fraud_str'].apply(lambda x: 1 if x == 'Yes' else 0)
            temp_df['id'] = temp_df['id'].astype('int64')
            
            write_mode = 'replace' if is_first_batch else 'append'
            
           
            temp_df[['id', 'is_fraud']].to_sql(
                table_name, 
                engine, 
                if_exists=write_mode,
                index=False
            )
            is_first_batch = False
            gc.collect()

except Exception as e:
   print(e)

--- الخطوة الثانية: تحميل الـ Labels من JSON إلى SQL ---
هبدأ أحمل ملف الـ JSON وأخزنه في جدول اسمه 'FraudLabels'...
العملية دي ممكن تاخد شوية وقت، من فضلك استنى...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 صف...
بعالج دفعة بحجم 100000 

In [8]:
import pandas as pd
from sqlalchemy import create_engine
import gc

server = 'localhost\\MSSQLSERVER2022'
database = 'Financial_Transactions'
connection_string = (
    f"mssql+pyodbc://{server}/{database}?"
    "driver=ODBC+Driver+17+for+SQL+Server"
)
engine = create_engine(connection_string)


sql_query = "SELECT * FROM dbo.Final_Analysis_View"
chunk_size = 1_000_000
processed_chunks = []
categorical_cols = ['merchant_state', 'gender', 'card_brand', 'card_type', 'merchant_category', 'merchant_city']

print("Starting to pull the final, clean data in chunks...")
for chunk in pd.read_sql_query(sql_query, engine, chunksize=chunk_size):
    print(f"Processing chunk of size {len(chunk)}...")
    
    for col in categorical_cols:
        if col in chunk.columns:
            chunk[col].fillna('Unknown', inplace=True)
            chunk[col] = chunk[col].astype('category')
            
    processed_chunks.append(chunk)
    gc.collect()
print("Combining all chunks...")
final_df = pd.concat(processed_chunks, ignore_index=True)

final_df.info()

Starting to pull the final, clean data in chunks...
Processing chunk of size 1000000...


D:\cuda\Temp\ipykernel_16944\948674309.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  chunk[col].fillna('Unknown', inplace=True)
D:\cuda\Temp\ipykernel_16944\948674309.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

Processing chunk of size 1000000...


D:\cuda\Temp\ipykernel_16944\948674309.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  chunk[col].fillna('Unknown', inplace=True)
D:\cuda\Temp\ipykernel_16944\948674309.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

Processing chunk of size 1000000...


...

D:\cuda\Temp\ipykernel_16944\948674309.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  chunk[col].fillna('Unknown', inplace=True)
D:\cuda\Temp\ipykernel_16944\948674309.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

Processing chunk of size 305915...


D:\cuda\Temp\ipykernel_16944\948674309.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  chunk[col].fillna('Unknown', inplace=True)
D:\cuda\Temp\ipykernel_16944\948674309.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

Combining all chunks...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 24 columns):
 #   Column             Dtype         
---  ------             -----         
 0   clean_id           int64         
 1   date               datetime64[ns]
 2   amount             float64       
 3   merchant_id        int64         
 4   merchant_city      object        
 5   merchant_state     object        
 6   mcc                int64         
 7   client_id          int64         
 8   current_age        int64         
 9   birth_year         int64         
 10  gender             category      
 11  yearly_income      float64       
 12  total_debt         float64       
 13  credit_score       int64         
 14  latitude           float64       
 15  longitude          float64       
 16  card_id            int64         
 17  card_brand         category      
 18  card_type          category      
 19  credit_limit       float64       
 20

D:\cuda\Temp\ipykernel_16944\948674309.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(processed_chunks, ignore_index=True)
